In [1]:
import sys
import os
from pathlib import Path

root = Path(os.getcwd()).parent
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
from datetime import datetime

In [3]:
route = pd.read_csv('aaa.csv', encoding='utf-8-sig', index_col=0)
route

,date,patNo,reason,route
0,2020-04-17,10779,해외 접촉 추정(미국),미국 LA 출발 아시아나항공(OZ201) 이용해 인천공항에 입국
1,2020-05-01,10789,해외 접촉 추정(터키),"인천국제공항(대한항공 KE8956, 좌석 55C)"
2,2020-05-01,10789,해외 접촉 추정(터키),공항
3,2020-04-28,10790,해외 접촉 추정(미국),"인천공항 입국(미국) *무증상, 동반 입국자 1명 음성 판정"
4,2020-05-02,10839,이태원 클럽 관련,"킹클럽(우사단로, 클럽), 퀸(우사단로, 주점) *도보"
...,...,...,...,...
6216,2020-08-16,15789,성북구 사랑제일교회 관련,", ~소독완료마스크 착용, 접촉자 없음<등록 강북구청>"
6217,2020-08-14,15833,확인 중,"소독완료마스크 착용, 접촉자 없음<등록 강동구청>"
6218,2020-08-14,15824,성북구 사랑제일교회 관련,"소독완료식사 외 마스크 착용, 접촉자 9명<등록 강동구청>"
6219,2020-08-15,15808,성북구 사랑제일교회 관련,"소독완료마스크 착용, 접촉자 없음<등록 강동구청>"


In [4]:
patient = pd.read_csv(os.path.join(root, 'data', 'PatientInfo2.csv'), encoding='utf-8-sig', index_col=0)
patient

,거주지,여행력,접촉력,퇴원현황,확진일,환자 번호
0,성동구,미국,해외 접촉 추정,퇴원,2020-04-13,10564
1,은평구,-,타시도 확진자 접촉,퇴원,2020-04-12,10524
2,도봉구,스페인,#9641 접촉,퇴원,2020-04-12,10528
3,송파구,미국,해외 접촉 추정,퇴원,2020-04-12,10537
4,성동구,미국,해외 접촉 추정,퇴원,2020-04-12,10522
...,...,...,...,...,...,...
2595,강북구,-,성북구 사랑제일교회 관련,퇴원,2020-08-17,15533
2596,성북구,-,성북구 사랑제일교회 관련,퇴원,2020-08-17,15587
2597,강북구,-,성북구 사랑제일교회 관련,퇴원,2020-08-17,15527
2598,송파구,-,용인시 우리제일교회 관련,퇴원,2020-08-17,15592


### 둘 다 확인 중인 경우

In [5]:
def check_check(reason):
    if '확인 중' in reason or '확인중' in reason:
        return True
    return False

In [6]:
# 둘 다 모두 확인 중인 경우 확인
numbers = []
for index1, row in patient.iterrows():
    number = row['환자 번호']
    route_rows = route.loc[route['patNo'] == number]
        
    count = 0
    for index2, route_row in route_rows.iterrows():
        if check_check(row['접촉력']) and check_check(route_row['reason']):
            count += 1
        
    if count == len(route_rows) and len(route_rows) != 0:
        numbers.append(number)
        
print(numbers)

# 삭제
for number in numbers:
    indices = route[route['patNo'] == number].index
    route = route.drop(indices, inplace=False)

route = route.reset_index(drop=True)
route

[11252, 11237, 11644, 11595, 11594, 11544, 11565, 11539, 11528, 11460, 11417, 11364, 11319, 11317, 11826, 11784, 11678, 12068, 12049, 12047, 11788, 12046, 12042, 12006, 11988, 12003, 11936, 11946, 11937, 12369, 12302, 12284, 12244, 12255, 12146, 12115, 12784, 12754, 12686, 12592, 12557, 12509, 12504, 12429, 12395, 12413, 13155, 12987, 12923, 12897, 12865, 12831, 12789, 13155, 12987, 12923, 12897, 12865, 12831, 12789]


,date,patNo,reason,route
0,2020-04-17,10779,해외 접촉 추정(미국),미국 LA 출발 아시아나항공(OZ201) 이용해 인천공항에 입국
1,2020-05-01,10789,해외 접촉 추정(터키),"인천국제공항(대한항공 KE8956, 좌석 55C)"
2,2020-05-01,10789,해외 접촉 추정(터키),공항
3,2020-04-28,10790,해외 접촉 추정(미국),"인천공항 입국(미국) *무증상, 동반 입국자 1명 음성 판정"
4,2020-05-02,10839,이태원 클럽 관련,"킹클럽(우사단로, 클럽), 퀸(우사단로, 주점) *도보"
...,...,...,...,...
5491,2020-08-16,15789,성북구 사랑제일교회 관련,", ~소독완료마스크 착용, 접촉자 없음<등록 강북구청>"
5492,2020-08-14,15833,확인 중,"소독완료마스크 착용, 접촉자 없음<등록 강동구청>"
5493,2020-08-14,15824,성북구 사랑제일교회 관련,"소독완료식사 외 마스크 착용, 접촉자 9명<등록 강동구청>"
5494,2020-08-15,15808,성북구 사랑제일교회 관련,"소독완료마스크 착용, 접촉자 없음<등록 강동구청>"


In [7]:
# 둘 다 확인 중이지만 다른 row에서는 그렇지 않은 경우
numbers = []
for index1, row in route.iterrows():
    number = row['patNo']
    patient_rows = patient.loc[patient['환자 번호'] == number]
    
    count = 0
    for index2, patient_row in patient_rows.iterrows():
        if check_check(patient_row['접촉력']) and check_check(row['reason']):
            count += 1
    
    if count > 0:
        numbers.append(number)
        
numbers = list(dict.fromkeys(numbers))
print(numbers)

# 다른 row에 있는 reason으로 변경
for number in numbers:
    unclear_route = route[route['patNo'] == number]
    reason = ''
    for row, elem in unclear_route.iterrows():
        if not check_check(elem['reason']):
            reason = elem['reason']
            
    indices = route[(route['patNo'] == number) & (route['reason'] == '확인 중')].index
    for index in indices:
        route.at[index, 'reason'] = reason
        
route

[11240, 11821, 11824, 12294]


,date,patNo,reason,route
0,2020-04-17,10779,해외 접촉 추정(미국),미국 LA 출발 아시아나항공(OZ201) 이용해 인천공항에 입국
1,2020-05-01,10789,해외 접촉 추정(터키),"인천국제공항(대한항공 KE8956, 좌석 55C)"
2,2020-05-01,10789,해외 접촉 추정(터키),공항
3,2020-04-28,10790,해외 접촉 추정(미국),"인천공항 입국(미국) *무증상, 동반 입국자 1명 음성 판정"
4,2020-05-02,10839,이태원 클럽 관련,"킹클럽(우사단로, 클럽), 퀸(우사단로, 주점) *도보"
...,...,...,...,...
5491,2020-08-16,15789,성북구 사랑제일교회 관련,", ~소독완료마스크 착용, 접촉자 없음<등록 강북구청>"
5492,2020-08-14,15833,확인 중,"소독완료마스크 착용, 접촉자 없음<등록 강동구청>"
5493,2020-08-14,15824,성북구 사랑제일교회 관련,"소독완료식사 외 마스크 착용, 접촉자 9명<등록 강동구청>"
5494,2020-08-15,15808,성북구 사랑제일교회 관련,"소독완료마스크 착용, 접촉자 없음<등록 강동구청>"


### 환자 정보에 등록되지 않았지만 다른 row에 reason이 있는 경우

In [8]:
# 환자 정보에 등록되지 않았지만 다른 row에 reason이 있는 경우
indices = route[route['reason'] == '확인 중'].index
indices_to_del = []
numbers = []
for index in indices:
    number = route.loc[index, 'patNo']
    reasons = patient[patient['환자 번호'] == number]['접촉력'].values
    
    if len(reasons) == 0:
        indices_to_del.append(index)
    else:
        numbers.append(number)
        route.at[index, 'reason'] = reasons[0]

numbers = list(dict.fromkeys(numbers))
print(numbers)
        
# route 모두 확인 중이고 환자 정보에도 등록되지 않은 경우 삭제
route = route.drop(indices_to_del, inplace=False)
route = route.reset_index(drop=True)
route

[11034, 11087, 11075, 11092, 11088, 11093, 11113, 11142, 11196, 11245, 11248, 11243, 11358, 11359, 11431, 11430, 11433, 11461, 11448, 11510, 11511, 11535, 11657, 11648, 11775, 11738, 11726, 11730, 11741, 11787, 11800, 11870, 12078, 12098, 12135, 12237, 12291, 12288, 12866, 13388, 15106, 15298]


,date,patNo,reason,route
0,2020-04-17,10779,해외 접촉 추정(미국),미국 LA 출발 아시아나항공(OZ201) 이용해 인천공항에 입국
1,2020-05-01,10789,해외 접촉 추정(터키),"인천국제공항(대한항공 KE8956, 좌석 55C)"
2,2020-05-01,10789,해외 접촉 추정(터키),공항
3,2020-04-28,10790,해외 접촉 추정(미국),"인천공항 입국(미국) *무증상, 동반 입국자 1명 음성 판정"
4,2020-05-02,10839,이태원 클럽 관련,"킹클럽(우사단로, 클럽), 퀸(우사단로, 주점) *도보"
...,...,...,...,...
5490,2020-08-17,15533,성북구 사랑제일교회 관련,소독완료접촉자 특정불가<등록 강북구청>
5491,2020-08-16,15789,성북구 사랑제일교회 관련,", ~소독완료마스크 착용, 접촉자 없음<등록 강북구청>"
5492,2020-08-14,15824,성북구 사랑제일교회 관련,"소독완료식사 외 마스크 착용, 접촉자 9명<등록 강동구청>"
5493,2020-08-15,15808,성북구 사랑제일교회 관련,"소독완료마스크 착용, 접촉자 없음<등록 강동구청>"


### '해외 접촉 추정' → 해외 접촉

In [9]:
indices = route[route['reason'].str.contains('해외 접촉|해외접촉', na=False)].index
for index in indices:
    route.at[indices, 'reason'] = '해외 접촉'
    
route[route['reason'].str.contains('해외 접촉', na=False)]

,date,patNo,reason,route
0,2020-04-17,10779,해외 접촉,미국 LA 출발 아시아나항공(OZ201) 이용해 인천공항에 입국
1,2020-05-01,10789,해외 접촉,"인천국제공항(대한항공 KE8956, 좌석 55C)"
2,2020-05-01,10789,해외 접촉,공항
3,2020-04-28,10790,해외 접촉,"인천공항 입국(미국) *무증상, 동반 입국자 1명 음성 판정"
436,2020-05-08,10862,해외 접촉,필리핀에서 인천공항 입국(마스크 착용)(해외입국자 전용리무진 이용)
...,...,...,...,...
5367,2020-06-21,13109,해외 접촉,카자흐스탄 공항 출발
5368,2020-06-22,13109,해외 접촉,인천공항 도착(마스크 착용)(자차 이용)
5375,2020-07-08,13362,해외 접촉,인천공항 입국
5376,2020-07-08,13362,해외 접촉,인천공항 입국


### 저장

In [10]:
path = os.path.join(root, 'data', 'modified_route.csv')
route.to_csv(path, encoding='utf-8-sig', index=False)